In [14]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import time

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu" )
print(device)

cuda


In [16]:

BASE_PATH = r"dataset augm"
image = []
labels = []

img_paths = []
img_class = [] 

for file in os.listdir(BASE_PATH):
    if file == "background":
        for elem in os.listdir(os.path.join(BASE_PATH, file)):
            img_paths.append(BASE_PATH+"/"+file+"/"+elem)
            img_class.append(0)
            
    else:
        for elem in os.listdir(os.path.join(BASE_PATH, file)):
            img_paths.append(BASE_PATH+"/"+file+"/"+elem)
            img_class.append(1)

print(img_paths[:5],img_class[:5])


['dataset augm/background/1.jpg', 'dataset augm/background/10.jpg', 'dataset augm/background/100.jpg', 'dataset augm/background/1000.jpg', 'dataset augm/background/10000.jpg'] [0, 0, 0, 0, 0]


In [17]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(img_paths, img_class, test_size=0.2, random_state=42)
print(x_train[:5])
print(np.sum(y_train))
print(np.sum(y_test))
print(y_train[:5])

['dataset augm/background/3669.jpg', 'dataset augm/background/8820.jpg', 'dataset augm/background/7463.jpg', 'dataset augm/lane/h1673032320037.png', 'dataset augm/background/14443.jpg']
8301
2099
[0, 0, 0, 1, 0]


In [18]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    # transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomVerticalFlip(p=0.5),
    # transforms.ColorJitter(contrast=(1),saturation=(0.5,1.5),hue=(-0.1,0.1)),
    # transforms.RandomRotation(degrees=15), 
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

In [19]:

class LaneDataset(Dataset):
    def __init__(self,img_data,img_class,transform=None) -> None:
        super().__init__()
        self.img_data=img_data
        self.img_class=img_class
        self.transform=transform

    def __len__(self):
        return len(self.img_data)
    
    def __getitem__(self, index):
        image=Image.open(self.img_data[index])
        if self.transform is not None:
            image=self.transform(image)
        label=torch.tensor(self.img_class[index])
        return image, label

In [20]:
trainDataset = LaneDataset(x_train,y_train,transform)
valdiationDataset = LaneDataset(x_test,y_test,transform)

batch_size=32
train_loader=DataLoader(trainDataset,batch_size=batch_size)
valdiation_loader=DataLoader(valdiationDataset,batch_size=batch_size)

In [21]:
import torch.nn as nn
import torch.nn.functional as F

class LaneClassifier(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1=nn.Conv2d(3,10,3)
        self.pool1=nn.MaxPool2d(2)
        self.conv2=nn.Conv2d(10,6,3)
        self.conv3=nn.Conv2d(6,4,3)
        self.conv4 = nn.Conv2d(4,2,1)
        self.drop = nn.Dropout(0.3)
        self.norm1 = nn.BatchNorm2d(10)
        self.norm2 = nn.BatchNorm2d(6)
        self.norm3 = nn.BatchNorm2d(4)
        self.norm4 = nn.BatchNorm2d(2)
        self.fc1=nn.Linear(2,2)
        
    
    def forward(self,x):
        x=F.relu(self.conv1(x))
        x=self.drop(x)
        x=self.norm1(x)
        x=F.relu(self.conv2(x))
        x=self.drop(x)
        x=self.norm2(x)
        x=F.relu(self.conv3(x))
        x=self.drop(x)
        x=self.norm3(x)
        x=self.conv4(x)
        print(x)
        x=F.max_pool2d(x, kernel_size=x.size()[2:])
        print(x)
        x=torch.squeeze(x)
        x=F.softmax(x,dim=1)
        return x
laneClassifier= LaneClassifier().to(device)
print(laneClassifier)

LaneClassifier(
  (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(6, 4, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(4, 2, kernel_size=(1, 1), stride=(1, 1))
  (drop): Dropout(p=0.3, inplace=False)
  (norm1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm2): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm3): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm4): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=2, out_features=2, bias=True)
)


In [22]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(laneClassifier.parameters(),lr=10**-4,weight_decay=10**-5)

In [23]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=False)

Using cache found in C:\Users\Aleman Ioan/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\Aleman Ioan\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Aleman Ioan\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [24]:
torch.cuda.empty_cache()
n_epochs = 50
print_every = 80
valid_loss_min = np.Inf
val_loss = []
val_acc = []
train_loss = []
train_acc = []
total_step = len(train_loader)
for epoch in range(1, n_epochs+1):
    running_loss = 0.0
    correct = 0
    total=0
    print(f'Epoch {epoch}\n')
    for batch_idx, (data_, target_) in enumerate(train_loader):
        data_, target_ = data_.to(device), target_.to(device)
        optimizer.zero_grad()
        start=time.time()

        outputs = laneClassifier(data_)
        end=time.time()
        target_=target_.long()
        loss = criterion(outputs,target_)
        loss.backward()
        optimizer.step()
        

        running_loss += loss.item()
        pred = torch.argmax(outputs, dim=1)
        correct += torch.sum(pred==target_).item()
        total += target_.size(0)
        if (batch_idx) % print_every == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Time:{:.4f}' 
                   .format(epoch, n_epochs, batch_idx, total_step, loss.item(),end-start))
    train_acc.append(100 * correct / total)
    train_loss.append(running_loss/total_step)
    print(f'\ntrain loss: {np.mean(train_loss):.4f}, train acc: {(100 * correct / total):.4f}')
    batch_loss = 0
    total_t=0
    correct_t=0
    with torch.no_grad():
        laneClassifier.eval()
        for data_t, target_t in (valdiation_loader):
            data_t, target_t = data_t.to(device), target_t.to(device)
            outputs_t = laneClassifier(data_t)
            target_t=target_t.long()
            loss_t = criterion(outputs_t, target_t)
            batch_loss += loss_t.item()
            pred_t = torch.argmax(outputs_t, dim=1)
            correct_t += torch.sum(pred_t==target_t).item()
            total_t += target_t.size(0)
        val_acc.append(100 * correct_t / total_t)
        val_loss.append(batch_loss/len(valdiation_loader))
        network_learned = batch_loss < valid_loss_min
        print(f'validation loss: {np.mean(val_loss):.4f}, validation acc: {(100 * correct_t / total_t):.4f}\n')
        # Saving the best weight
        if network_learned:
            valid_loss_min = batch_loss
            torch.save(laneClassifier.state_dict(), 'model_classification_tutorial.pt')
            print('Detected network improvement, saving current model')
    laneClassifier.train()

Epoch 1

tensor([[[[-3.4725e-01, -9.9189e-01, -3.1279e-01,  ..., -2.7761e-01,
            1.9714e-02, -1.3163e-01],
          [-1.6709e+00,  1.6314e+00, -8.5967e-01,  ..., -3.4725e-01,
           -2.6700e+00,  3.7239e-01],
          [ 1.8263e+00,  9.7004e-01, -3.4725e-01,  ...,  1.6898e+00,
           -3.4725e-01, -1.3215e+00],
          ...,
          [-1.1885e+00, -3.1162e-01,  1.5952e-01,  ..., -2.9764e-02,
           -2.7315e+00, -1.2804e+00],
          [ 2.4489e-01, -2.1288e+00,  1.7973e+00,  ..., -2.2670e+00,
           -1.0368e-01, -3.7050e-01],
          [-3.1537e+00,  2.3767e-01, -7.4801e-01,  ..., -3.4725e-01,
            6.7015e-02,  2.7543e-01]],

         [[ 7.1699e-01,  2.5669e-01,  7.0216e-01,  ...,  1.4812e-01,
            2.1555e-02, -1.0443e+00],
          [-7.9558e-01,  1.5071e-01, -2.9305e-01,  ...,  7.1699e-01,
           -1.3835e+00,  4.0730e-01],
          [ 5.7682e-01, -3.0289e-01,  7.1699e-01,  ..., -1.5966e-01,
            7.1699e-01,  2.1335e-02],
          .

KeyboardInterrupt: 

In [ ]:
plt.subplot(1, 2, 1)
plt.plot(val_acc, label="val_acc")
plt.plot(train_acc, label="train_acc")
plt.legend(loc="upper left")
plt.subplot(1, 2, 2)
plt .plot(val_loss, label="val_loss")
plt .plot(train_loss, label="train_loss")
plt.legend(loc="upper left")